# Python Exception Handling — Foundations to Advanced

**Focus:** Try/except blocks, built-in & user-defined exceptions, and advanced error-handling strategies.

> Run cells with **Shift+Enter**. Modify code to observe different outcomes.

## Agenda
- Exception Handling in Python — Agenda  
- What is an Exception in Python  
- Handling Exception with Syntax in Python  
- Types of Exception Handling in Python — Part 1 & 2  
- Try/Except in Python (variants: else, finally, multiple except)  
- Errors vs Exceptions  
- Built-in Exceptions — Part 1 & Part 2  
- User-defined Exceptions  
- Advantages & Disadvantages of Exception Handling in Python  
- **Summary** + **9 Coding Exercises** (Beginner ➜ Advanced)

## What is Exception in Python

An **exception** is an event that disrupts normal program flow during execution (e.g., file not found, division by zero).  
Python signals exceptions by **raising** them; you **handle** them to recover or fail gracefully.

In [ ]:
# A quick example that raises an exception
def divide(a, b):
    return a / b

try:
    print(divide(10, 0))
except ZeroDivisionError as e:
    print("Caught:", e.__class__.__name__, "-", e)

## Handling Exception with Syntax in Python

Basic forms:
- `try: ... except ExceptionType as e: ...`
- `try: ... except ... else: ...` (runs only if no exception)
- `try: ... finally: ...` (always runs)
- `try: ... except ... else: ... finally: ...`
- Multiple except clauses

In [ ]:
def read_int(s):
    try:
        x = int(s)
    except ValueError as e:
        print("Not an integer:", e)
        return None
    else:
        print("Parsed successfully:", x)
        return x
    finally:
        # Always executes
        print("done read_int")
        
read_int("42")
read_int("forty-two")

## Types of Exception Handling — Part 1

**Narrow vs Broad catching**
- Prefer catching **specific** exceptions you expect.
- Avoid `except Exception:` unless you re-raise or log, and you truly need a broad net.

In [ ]:
def safe_get(d, key):
    try:
        return d[key]
    except KeyError:                 # specific expected failure
        return None

print(safe_get({"a": 1}, "a"))
print(safe_get({"a": 1}, "b"))

## Types of Exception Handling — Part 2

**EAFP vs LBYL**
- **EAFP** (Easier to Ask Forgiveness than Permission): try/except; Pythonic.
- **LBYL** (Look Before You Leap): check conditions before action.

In [ ]:
# EAFP
def get_len(obj):
    try:
        return len(obj)
    except TypeError:
        return 0

# LBYL
def get_len_lbyl(obj):
    return len(obj) if hasattr(obj, "__len__") else 0

print(get_len([1,2,3]), get_len(10))
print(get_len_lbyl([1,2,3]), get_len_lbyl(10))

## Try Except in Python — Variations

- Multiple `except` blocks
- Grouping exceptions in a tuple
- `else` (only when no exception)
- `finally` (always run; close resources)

In [ ]:
def process(s):
    try:
        n = int(s)
        print("as int:", n)
    except (ValueError, TypeError) as e:
        print("cannot convert:", e)
    else:
        print("success branch")
    finally:
        print("cleanup (e.g., close file handles here)")

process("100")
process("abc")

## Errors and Exceptions in Python

- **SyntaxError** occurs at parse time (before code runs).
- **Exceptions** occur at runtime (e.g., `FileNotFoundError`).  
Try running the next cell to see a runtime exception; the SyntaxError example is shown as a comment.

In [ ]:
# SyntaxError example (don't run, illustrative):
# if True print("hello")   # 👈 missing colon would cause SyntaxError at import/parse time

# Runtime exception demo:
try:
    open("/path/that/does/not/exist.txt")
except FileNotFoundError as e:
    print("Runtime exception:", e.__class__.__name__, "-", e)

## Built-in Exceptions — Part 1

Common ones:
- `ValueError`, `TypeError`, `IndexError`, `KeyError`, `AttributeError`
- `ZeroDivisionError`, `FileNotFoundError`, `PermissionError`
- `AssertionError`, `ImportError`, `ModuleNotFoundError`

In [ ]:
tests = [
    ("ValueError", lambda: int("x")),
    ("TypeError", lambda: "a" + 1),
    ("IndexError", lambda: [][0]),
    ("KeyError", lambda: {}["missing"]),
    ("AttributeError", lambda: (10).missing),
]

for name, fn in tests:
    try:
        fn()
    except Exception as e:
        print(f"{name:>14}: caught {e.__class__.__name__} -> {e}")

## Built-in Exceptions — Part 2

Advanced / less common:
- `StopIteration` (iterator protocol), `RuntimeError`
- `OSError` family (`FileExistsError`, `IsADirectoryError`, etc.)
- `TimeoutError`, `ConnectionError` (network)
- `MemoryError`, `RecursionError`

In [ ]:
import sys

def cause_recursion_error(n):
    if n == 0:
        return 1
    return n * cause_recursion_error(n-1)

try:
    cause_recursion_error(10_000)  # large enough to overflow recursion limit
except RecursionError as e:
    print("Caught:", e.__class__.__name__)

## User-defined Exceptions in Python

Create custom exceptions by subclassing `Exception` (or a specific built-in).  
Organize a small **exception hierarchy** for your domain.

In [ ]:
class AppError(Exception):
    """Base class for app-specific errors."""

class ConfigError(AppError):
    pass

class ValidationError(AppError):
    def __init__(self, field, message):
        super().__init__(f"{field}: {message}")
        self.field = field
        self.message = message

def load_config(cfg):
    if "db" not in cfg:
        raise ConfigError("Missing 'db' section")
    return True

def validate_user(user):
    if not user.get("email"):
        raise ValidationError("email", "required")

try:
    load_config({})
except AppError as e:
    print("Config error caught:", e)

try:
    validate_user({"name": "Luna"})
except ValidationError as e:
    print("Validation caught:", e.field, "-", e.message)

## Advanced Patterns: Chaining, Re-raise, Logging, Context Managers

- **Re-raise** with `raise` inside `except` to bubble up after logging.
- **Exception chaining** with `raise NewError() from old_error` for clearer cause.
- Suppress chaining via `from None` to hide internal details when appropriate.
- Use **context managers** (`with`) for reliable cleanup.

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)

def parse_price(s: str) -> float:
    try:
        return float(s.strip().replace("$", ""))
    except ValueError as e:
        logging.exception("bad price: %r", s)
        # re-raise with context
        raise ValueError(f"Invalid price: {s}") from e

try:
    parse_price("$12.34")
    parse_price("twelve dollars")
except ValueError as e:
    print("Handled upstream:", e)

In [ ]:
# Context manager example (ensures file is closed even if exceptions occur)
from tempfile import NamedTemporaryFile

with NamedTemporaryFile(mode="w+t", delete=True) as f:
    f.write("hello\n")
    f.seek(0)
    print("file says:", f.read().strip())
# file auto-closed here

## ExceptionGroup & `except*` (Python 3.11+)

`ExceptionGroup` lets you raise & handle multiple exceptions concurrently (e.g., in parallel tasks).  
Use `except*` to catch subsets.

In [ ]:
def make_group():
    excs = [ValueError("bad value"), TypeError("bad type")]
    raise ExceptionGroup("two problems", excs)

try:
    make_group()
except* ValueError as eg:
    print("handled ValueError group:", eg)
except* TypeError as eg:
    print("handled TypeError group:", eg)

## Advantages and Disadvantages of Exception Handling in Python

**Advantages**
- Clear separation of error path from normal path
- Encourages robust, recoverable code
- Works well with context managers and logging

**Disadvantages / Pitfalls**
- Overbroad catching can hide bugs
- Swallowing exceptions without logging = silent failures
- Control flow via exceptions in hot paths can impact performance
- Relying on `__del__` for cleanup is unsafe; prefer `with` or `try/finally`

## Exception Handling in Python — Summary

- Use **specific** exceptions where possible; log before re-raising if needed  
- Prefer **EAFP** with clear try blocks and tight **except** scopes  
- Leverage `else` for success-only logic and `finally` for cleanup  
- Build a **custom exception hierarchy** for your domain  
- Utilize **context managers** and consider **ExceptionGroup** for concurrency

---

# 🧪 9 Coding Exercises

**Instructions:** Implement the `TODO` parts. Run tests below each exercise to check your solution.

### Exercise 1 (Beginner): Safe Int Parse

In [ ]:
def safe_int(s, default=None):
    # TODO: return int(s); on ValueError, return default
    try:
        return int(s)
    except ValueError:
        return default

assert safe_int("42") == 42
assert safe_int("x", 0) == 0
print("✓ Exercise 1 passed")

### Exercise 2 (Beginner): Open File Safely

In [ ]:
def read_text(path):
    # TODO: open path and return content; on FileNotFoundError, return ""
    try:
        with open(path, "r", encoding="utf-8") as f:
            return f.read()
    except FileNotFoundError:
        return ""

print("✓ Exercise 2 sample run (file may not exist in this environment)")

### Exercise 3 (Beginner): Fallback Division

In [ ]:
def div(a, b, fallback=None):
    # TODO: return a/b; on ZeroDivisionError or TypeError return fallback
    try:
        return a / b
    except (ZeroDivisionError, TypeError):
        return fallback

assert div(10, 2) == 5
assert div(10, 0, "inf") == "inf"
assert div(10, "x", None) is None
print("✓ Exercise 3 passed")

### Exercise 4 (Intermediate): Validate Dict Field

In [ ]:
class FieldError(Exception): ...
def require_field(d, key):
    # TODO: return d[key]; on KeyError raise FieldError(key)
    try:
        return d[key]
    except KeyError as e:
        raise FieldError(key) from e

try:
    require_field({"a": 1}, "b")
except FieldError as e:
    print("caught FieldError:", e)

### Exercise 5 (Intermediate): Retry with Backoff

In [ ]:
import time

def retry(op, retries=3, delay=0.1):
    """Run op(); retry on Exception up to 'retries' with 'delay' seconds between."""
    # TODO: implement retry loop; re-raise after exhausting attempts
    last_exc = None
    for i in range(retries):
        try:
            return op()
        except Exception as e:
            last_exc = e
            if i < retries - 1:
                time.sleep(delay)
            else:
                raise

# Demo with flaky operation
count = 0
def flaky():
    global count
    count += 1
    if count < 2:
        raise RuntimeError("not yet")
    return "ok"

print("retry result:", retry(flaky, retries=3, delay=0.01))

### Exercise 6 (Intermediate): Context Manager for Suppression

In [ ]:
from contextlib import contextmanager

@contextmanager
def suppress(*exc_types):
    # TODO: implement a context manager that suppresses given exception types
    try:
        yield
    except exc_types:
        pass

# Test
with suppress(ZeroDivisionError):
    1/0
print("✓ Exercise 6 passed")

### Exercise 7 (Advanced): Normalize Price

In [ ]:
def normalize_price(s):
    """Return float price from strings like '$12.30', ' 12.30 ', '12,30' (comma -> dot).
    On error, raise ValueError with a helpful message (chain original).
    """
    # TODO
    try:
        s2 = s.strip().replace("$", "").replace(",", ".")
        return float(s2)
    except Exception as e:
        raise ValueError(f"Invalid price: {s!r}") from e

assert abs(normalize_price("$12.30") - 12.3) < 1e-9
assert abs(normalize_price(" 12,30 ") - 12.3) < 1e-9
try:
    normalize_price("twelve")
except ValueError as e:
    print("✓ Exercise 7 raised as expected:", e)

### Exercise 8 (Advanced): Validate Email with Custom Error

In [ ]:
import re
class EmailError(Exception): ...
def validate_email(s):
    # TODO: use a simple regex; on failure raise EmailError from the internal cause (if any)
    pat = re.compile(r"^[\w.-]+@[\w.-]+\.[A-Za-z]{2,}$")
    if not pat.match(s or ""):
        raise EmailError(f"Invalid email: {s!r}")
    return True

assert validate_email("a.b@example.com") is True
try:
    validate_email("bad@com")
except EmailError as e:
    print("✓ Exercise 8 raised:", e)

### Exercise 9 (Advanced): Aggregate Errors with ExceptionGroup

In [ ]:
def check_many(values):
    excs = []
    for v in values:
        try:
            if int(v) < 0:
                raise ValueError(f"negative: {v}")
        except Exception as e:
            excs.append(e)
    if excs:
        raise ExceptionGroup("validation", excs)

try:
    check_many(["1", "-2", "x", "3"])  # ValueError + ValueError(from int('x'))
except* ValueError as eg:
    print("handled ValueErrors:", eg)
except* Exception as eg:
    print("other errors:", eg)